# Analyse et nettoyage des données Immatriculation

## Import

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql.types import BooleanType, FloatType
from pyspark.sql.functions import trim, lower, upper, col, count, regexp_replace, max, min, when,avg, round, rand

## Session spark

In [ ]:
spark = SparkSession.builder\
    .appName("CleanImmatriculation")\
    .enableHiveSupport()\
    .getOrCreate()

## Import des données

In [ ]:
spark.sparkContext.setLogLevel("OFF")
spark.catalog.clearCache()
spark.sql("USE concessionnaire")

df_immat = spark.sql("SELECT * FROM immatriculations_ext")
df_client = spark.sql("SELECT * FROM clients")


In [ ]:
df_immat.show()
df_client.show()
df_client.count()

## Analyse

### Casse
- Renommage de la colonne "nom" en "modele" dans la table **immatriculation**
- Normalisation des marques de la table **immatriculation**
- Normalisation des modèles de la table **immatriculation**
- Suppression de la 1ère ligne de la table **clients**

#### Traitement Immatriculation

In [ ]:
#Renommage de la colonne "nom" en "modele"
df_immat = df_immat.withColumnRenamed("nom", "modele")

In [ ]:
df_immat.printSchema()

In [ ]:
#Normalisation des casses
df_immat = df_immat.withColumn("marque", lower(trim(col("marque"))))
df_immat = df_immat.withColumn("modele", lower(trim(col("modele"))))
df_immat = df_immat.withColumn("immatriculation", upper(trim(col("immatriculation"))))

In [ ]:
df_immat.show(n=5)

#### Traitemment Client

In [ ]:
#Suppression de la 1ère ligne de la table clients
df_client = df_client.filter(df_client['immatriculation'] != 'immatriculation')

#Normalisation de situation familiale et immatriculation
df_client = df_client.withColumn("situationfamiliale", lower(trim(col("situationfamiliale"))))
df_client = df_client.withColumn("immatriculation", upper(trim(col("immatriculation"))))

In [ ]:
df_client.show(n=3)

### Recherche des caractères spéciaux

- Recherche du symbole "�"
- Recherche du caractère "ã©"

In [ ]:
df_client.printSchema()

In [ ]:
#Recherche du symbole "�"

#table immatriculation
df_search_special_char_immat = df_immat.filter(
    col("immatriculation").like("%�%") | 
    col("marque").like("%�%") | 
    col("modele").like("%�%") | 
    col("longueur").like("%�%") |
    col("couleur").like("%�%") | 
    col("occasion").like("%�%")
)

#table clients
df_search_special_char_client = df_client.filter(
    col("sexe").like("%�%") |
    col("immatriculation").like("%�%")
)

value_client = df_client.groupBy("situationfamiliale").count()


In [ ]:
df_search_special_char_immat.show()
df_search_special_char_client.show()
value_client.show()

In [ ]:
#Recherche du caractère "ã©"

#table immatriculation
df_search_special_char_2_immat = df_immat.filter(
    col("immatriculation").like("%ã©%") | 
    col("marque").like("%ã©%") | 
    col("modele").like("%ã©%") | 
    col("longueur").like("%ã©%") |
    col("couleur").like("%ã©%") | 
    col("occasion").like("%ã©%")
)

#table clients
df_search_special_char_2_client = df_client.filter(
    col("sexe").like("%ã©%") | 
    col("situationfamiliale").like("%ã©%") | 
    col("immatriculation").like("%ã©%")
)

In [ ]:
df_search_special_char_2_immat.show()
df_search_special_char_2_client.show()

### Correction des caractères spéciaux

- Correction du symbole "�" dans la colonne "longueur".


In [ ]:
#Correction du symbole "�" dans la colonne "longueur"

df_immat = df_immat.withColumn("longueur", regexp_replace(col("longueur"), "�", "e"))

df_client = df_client.withColumn("situationfamiliale", regexp_replace(col("situationfamiliale"), "�", "e"))


In [ ]:

df_search_special_char_immat = df_immat.filter(
    col("immatriculation").like("%�%") | 
    col("marque").like("%�%") | 
    col("modele").like("%�%") | 
    col("longueur").like("%�%") |
    col("couleur").like("%�%") | 
    col("occasion").like("%�%")
)

value_client = df_client.groupBy("situationfamiliale").count()

value_client.show()

df_search_special_char_immat.show()

## Recherche et correction des Null

- Recherche des null

In [ ]:

#Recherche des null

# Liste des colonnes du DataFrame
colonnes_immat = df_immat.columns

# Filtrer pour conserver uniquement les lignes où au moins une colonne est nulle
df_immat_nulls = df_immat.filter(
    sum(col(colonne_immat).isNull().cast("int") for colonne_immat in colonnes_immat) > 0
)

# Liste des colonnes du DataFrame
colonnes_client = df_client.columns

# Filtrer pour conserver uniquement les lignes où au moins une colonne est nulle
df_client_nulls = df_client.filter(
    sum(col(colonne_client).isNull().cast("int") for colonne_client in colonnes_client) > 0
)

In [ ]:
# Afficher les lignes avec des valeurs nulles
df_immat_nulls.show()
df_client_nulls.show()

## Traitement des nulls dans la table **age**

- Analyse des valeurs inférieur à 18.
- Mettre à null les valeurs inférieures à 18 ans.
- Calculer la médiane.
- Remplacer les nulls par la médiane.

In [ ]:
#Analyse des valeurs inférieur à 18.

# Ajouter une colonne qui spécifie si l'âge est inférieur à 18
df_client_age = df_client.withColumn("age_below_18", when(col("age") < 18, 1).otherwise(0))

# Compter les lignes où "age_below_18" est 1
count_grouped = df_client_age.groupBy("age_below_18").count()



In [ ]:
# Afficher les résultats
count_grouped.show()


In [ ]:
#Mettre à null les valeurs inférieures à 18 ans.
df_client = df_client.withColumn("age", when(col("age") == -1, None).otherwise(col("age")))
df_client = df_client.withColumn("age", when(col("age") == 0, None).otherwise(col("age")))

#Calculer la médiane.
mediane = df_client.approxQuantile("age", [0.5], 0.01)[0]

#Remplacer les nulls par la médiane.
df_client = df_client.withColumn("age", when(col("age").isNull(), mediane).otherwise(col("age")))

In [ ]:
# Afficher la médiane
print("Médiane:", mediane)

value_client = df_client.groupBy("age").count()
value_client.show()

In [ ]:
#Analyse des données hors domaine

valeurs_hors_champs_min = df_client.filter(col("age") < 18)
valeurs_hors_champs_max = df_client.filter(col("age") > 84)

# Comptage des valeurs hors champ
valeurs_hors_champs_min.count()

## Traitement de **taux**

- Nettoyer situation familiale
    - Fusionner seul/seule en Celibataire
    - Remplacer les null/? par N/D
    - Analyser les différentes proportions
    - Supprimer les N/D si les proportions le permettent

- Faire la moyenne des taux par valeur de situation familiale
- Analyser la pertinence de ces moyennes
- Remplacer les nulls
- Analyse des données hors domaine
- Correction des valeurs hors domaine (Création d'une colonne "taux_eligible")

In [ ]:
#Nettoyer situation familiale

#Fusionner seul/seule en Celibataire
df_client = df_client.withColumn("situationfamiliale", regexp_replace(col("situationfamiliale"), "seule", "celibataire"))
df_client = df_client.withColumn("situationfamiliale", regexp_replace(col("situationfamiliale"), "seul", "celibataire"))
df_client = df_client.withColumn("situationfamiliale", regexp_replace(col("situationfamiliale"), "divorcee", "divorce(e)"))

# Remplacer "N/D" par null dans la colonne situationfamiliale
df_client = df_client.withColumn(
    "situationfamiliale",
    when(col("situationfamiliale") == None, "n/d").otherwise(col("situationfamiliale"))
)
df_client = df_client.withColumn(
    "situationfamiliale",
    when(trim(col("situationfamiliale")) == "", "n/d").otherwise(col("situationfamiliale"))
)
# Remplacer "?" par "N/D" dans la colonne situationfamiliale
df_client = df_client.withColumn(
    "situationfamiliale",
    regexp_replace(col("situationfamiliale"), r"\?", "n/d")
)


In [ ]:
value_client = df_client.groupBy("situationfamiliale").count()
value_client.show()

In [ ]:
#Analyser les différentes proportions

df_counts = df_client.groupBy("situationfamiliale").agg(count("*").alias("count"))
total_count = df_counts.selectExpr("SUM(count) as total").collect()[0]["total"]

df_percentages = df_counts.withColumn("percentage", (col("count") / total_count) * 100)

# Collecter les données dans un format utilisable pour Matplotlib
data = df_percentages.collect()
labels = [row["situationfamiliale"] if row["situationfamiliale"] else "Inconnue" for row in data]
sizes = [row["percentage"] for row in data]
df_percentages.show()

# Étape 2 : Créer le diagramme camembert
plt.figure(figsize=(8, 8))
plt.pie(
    sizes, 
    labels=labels, 
    autopct="%1.1f%%", 
    startangle=90, 
    colors=plt.cm.Paired.colors
)

plt.axis("equal")

plt.show()

In [ ]:
#Supprimer les N/D car les proportions le permettent
df_client= df_client.filter(col("situationfamiliale") != "n/d")


In [ ]:
df_counts = df_client.groupBy("situationfamiliale").agg(count("*").alias("count"))
total_count = df_counts.selectExpr("SUM(count) as total").collect()[0]["total"]

df_percentages = df_counts.withColumn("percentage", (col("count") / total_count) * 100)

# Collecter les données dans un format utilisable pour Matplotlib
data = df_percentages.collect()
labels = [row["situationfamiliale"] if row["situationfamiliale"] else "Inconnue" for row in data]
sizes = [row["percentage"] for row in data]
df_percentages.show()

In [ ]:
# Faire la moyenne des taux par valeur de situation familiale
mediane_taux = df_client.groupBy("situationfamiliale").agg(
    F.expr("percentile_approx(taux, 0.5, 100)").alias("mediane_taux")
)


In [ ]:
#Analyser la pertinence de ces moyennes

# Récupération des moyennes pour afficher dans un histogramme
data = mediane_taux.collect()
categories = [row["situationfamiliale"] for row in data]
mediane_taux = [row["mediane_taux"] for row in data]

plt.figure(figsize=(10, 6))
bars = plt.bar(categories, mediane_taux, color='skyblue')

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}', 
             ha='center', va='bottom', fontsize=10)

plt.xlabel("Situation Familiale")
plt.ylabel("Taux Médian")
plt.title("Taux Médian par Situation Familiale")
plt.xticks(rotation=45)
plt.tight_layout() 
plt.show()


In [ ]:
# Remplacer les nulls

mediane_taux = df_client.approxQuantile("taux", [0.5], 0.01)[0]
df_client = df_client.withColumn("taux", when(col("taux").isNull(), mediane).otherwise(col("taux")))

In [ ]:
print("Médiane:", mediane_taux)

In [ ]:
#Analyse des données hors domaine

valeurs_hors_champs_min = df_client.filter(col("taux") < 544)
valeurs_hors_champs_max = df_client.filter(col("taux") > 74185)

# Comptage des valeurs hors champ
valeurs_hors_champs_min.count()



In [ ]:
valeurs_hors_champs_max.count()

In [ ]:
#Correction des valeurs hors domaine (Création d'une colonne "taux_eligible")
df_client = df_client.withColumn(
    "taux_eligible",
    when((col("taux") >= 544) & (col("taux") <= 74185), True).otherwise(False)
)

In [ ]:
# Affichage des résultats
df_client.show()

## Traitement des null dans **nbenfantacharge**

- Faire la moyenne des enfants à charges par valeur de situation familiale
- Analyser la pertinence de ces moyennes
- Remplacer les nulls

In [ ]:
# Faire la moyenne des enfants à charges par valeur de situation familiale
moyenne_enfant = df_client.groupBy("situationfamiliale").agg(
    round(avg("nbenfantacharge"), 0).cast("int").alias("moyenne_enfant")
)

In [ ]:
#Analyser la pertinence de ces moyennes

# Récupération des moyennes pour afficher dans un histogramme
data = moyenne_enfant.collect()
categories = [row["situationfamiliale"] for row in data]
moyenne_enfant = [row["moyenne_enfant"] for row in data]

plt.figure(figsize=(10, 6))
bars = plt.bar(categories, moyenne_enfant, color='skyblue')
plt.xlabel("Situation Familiale")
plt.ylabel("Nombre enfant moyen")
plt.title("Nombre d'enfant moyen par Situation Familiale")
plt.xticks(rotation=45)
plt.tight_layout() 
plt.show()


In [ ]:
# Filtrer les données pour les célibataires et compter les valeurs non nulles de 'nbenfantacharge'

df_celibataire = df_client.filter(col("situationfamiliale") == "celibataire")
value_client = df_celibataire.groupBy("nbenfantacharge").count()

In [ ]:
value_client.show()

In [ ]:
df_client = df_client.withColumn(
    "nbenfantacharge",
    when(
        (col("situationfamiliale") == "celibataire") & (col("nbenfantacharge").isNull() | (col("nbenfantacharge") == -1)),
        0
    ).otherwise(col("nbenfantacharge"))
)

df_client = df_client.withColumn(
    "nbenfantacharge",
    when(
        (col("situationfamiliale").isin("marie(e)", "divorce(e)", "en couple")) & 
        (col("nbenfantacharge").isNull() | (col("nbenfantacharge") == -1)),
        2
    ).otherwise(col("nbenfantacharge"))
)


In [ ]:
value_client = df_client.groupBy("nbenfantacharge").count()
value_client.show()

## Traitement des null dans **sexe**

- Fusionner F/F�minin/Femme en F
- Fusionner H/Masculin/Hommme en H
- Remplacer les null/? par N/D
- Analyser les différentes proportions
- Supprimer les N/D si les proportions le permettent

In [ ]:
value_client = df_client.groupBy("sexe").count()
value_client.show()

In [ ]:
#Fusionner F/F�minin/Femme en F
df_client = df_client.withColumn("sexe", regexp_replace(col("sexe"), "Femme", "f"))
df_client = df_client.withColumn("sexe", regexp_replace(col("sexe"), "F�minin", "f"))
df_client = df_client.withColumn("sexe", regexp_replace(col("sexe"), "F", "f"))

#Fusionner H/M/Masculin/Hommme en H
df_client = df_client.withColumn("sexe", regexp_replace(col("sexe"), "Masculin", "h"))
df_client = df_client.withColumn("sexe", regexp_replace(col("sexe"), "M", "h"))
df_client = df_client.withColumn("sexe", regexp_replace(col("sexe"), "Homme", "h"))
df_client = df_client.withColumn("sexe", regexp_replace(col("sexe"), "H", "h"))

#Remplacer les null/? par N/D
df_client = df_client.withColumn("sexe", when(col("sexe") == None, "n/d").otherwise(col("sexe")))
df_client = df_client.withColumn("sexe",when(trim(col("sexe")) == "", "n/d").otherwise(col("sexe")))
df_client = df_client.withColumn("sexe",when(trim(col("sexe")) == "N/D", "n/d").otherwise(col("sexe")))
df_client = df_client.withColumn("sexe",regexp_replace(col("sexe"), r"\?", "n/d"))

In [ ]:
value_client = df_client.groupBy("sexe").count()
value_client.show()

In [ ]:
#Analyser les différentes proportions

df_counts = df_client.groupBy("sexe").agg(count("*").alias("count"))
total_count = df_counts.selectExpr("SUM(count) as total").collect()[0]["total"]

df_percentages = df_counts.withColumn("percentage", (col("count") / total_count) * 100)

# Collecter les données dans un format utilisable pour Matplotlib
data = df_percentages.collect()
labels = [row["sexe"] if row["sexe"] else "Inconnue" for row in data]
sizes = [row["percentage"] for row in data]
df_percentages.show()

# Étape 2 : Créer le diagramme camembert
plt.figure(figsize=(8, 8))
plt.pie(
    sizes, 
    labels=labels, 
    autopct="%1.1f%%", 
    startangle=90, 
    colors=plt.cm.Paired.colors
)

plt.axis("equal")

plt.show()

In [ ]:
#Supprimer les N/D si les proportions le permettent

df_client= df_client.filter(col("sexe") != "n/d")

In [ ]:
value_client = df_client.groupBy("sexe").count()
value_client.show()

## Traitement des null dans **deuxiemevoiture**

- Calculer le pourcentage true/false
- Remplacer les null en true et false en fonction des pourcentage de chacun.

In [ ]:
value_client = df_client.groupBy("deuxiemevoiture").count()
value_client.show()

In [ ]:
#Calculer le pourcentage true/false

df_filtered = df_client.filter(col("deuxiemevoiture").isNotNull())
df_counts = df_filtered.groupBy("deuxiemevoiture").agg(count("*").alias("count"))
total_count = df_counts.selectExpr("SUM(count) as total").collect()[0]["total"]

df_percentages = df_counts.withColumn("percentage", (col("count") / total_count) * 100)

df_percentages.show()

In [ ]:
#Remplacer les null en true et false en fonction des pourcentage de chacun.

df_nulls = df_client.filter(col("deuxiemevoiture").isNull())

# Ajouter une colonne aléatoire et assigner 'true' ou 'false' selon les proportions
df_nulls_replaced = df_nulls.withColumn(
    "deuxiemevoiture",
    when(rand() < 0.13, True).otherwise(False)
)

# Filtrer les lignes sans 'null' dans 'deuxiemevoiture'
df_non_nulls = df_client.filter(col("deuxiemevoiture").isNotNull())

# Combiner les deux DataFrames
df_client = df_non_nulls.union(df_nulls_replaced)


In [ ]:
df_counts = df_client.groupBy("deuxiemevoiture").agg(count("*").alias("count"))
total_count = df_counts.selectExpr("SUM(count) as total").collect()[0]["total"]

df_percentages = df_counts.withColumn("percentage", (col("count") / total_count) * 100)

df_percentages.show()

## Traitement des null dans **immatriculation**

- Compter les null
- Compter les doublons
- Analyser les doublons
- Suppression des doublons

In [ ]:
#Compter les null
df_null_counts = df_client.select(
    count(col("immatriculation")).alias("non_null"),
    count(when(col("immatriculation").isNull(), 1)).alias("null")
)

# Afficher le résultat
df_null_counts.show()

In [ ]:
#Compter les doublons

df_duplicates = (
    df_client.groupBy("immatriculation")
    .agg(count("*").alias("count"))
    .filter(col("count") > 1)  # Filtrer les doublons (où le compte est > 1)
)

# Afficher les résultats
df_duplicates.show()

#Analyser les doublons

duplicate_values = (
    df_client.groupBy("immatriculation")
    .agg(count("*").alias("count"))
    .filter(col("count") > 1)  # Garde seulement les doublons
    .select("immatriculation")  # Récupère uniquement la colonne des doublons
)

df_duplicate_rows = df_client.join(
    duplicate_values, on="immatriculation", how="inner"
)

# Afficher les lignes des doublons
df_duplicate_rows.show()

In [ ]:
#Suppression des doublons
df_client= df_client.dropDuplicates(["immatriculation"])

In [ ]:
df_duplicates = (
    df_client.groupBy("immatriculation")
    .agg(count("*").alias("count"))
    .filter(col("count") > 1)  # Filtrer les doublons (où le compte est > 1)
)

# Afficher les résultats
df_duplicates.show()


## Modification du type des colonnes occasion

- Recherche des valeurs d'occasion
- Changer le type de la colonne

In [ ]:
#Recherche des valeurs d'occasion
value_count_occasion = df_immat.groupBy("occasion").count()

value_count_occasion.show()

In [ ]:
#Changer le type de la colonne
df_immat = df_immat.withColumn("occasion", df_immat["occasion"].cast(BooleanType()))

df_immat.printSchema()

## Recherche de valeurs aberrantes

- Modéliser les valeurs dans un diagramme à moustaches pour mettre en avant les valeurs aberrantes.
- Afficher les valeurs suspectes.

In [ ]:
#Modéliser les valeurs dans un diagramme à moustaches pour mettre en avant les valeurs aberrantes.

# 2. Sélectionner uniquement les colonnes numériques nécessaires

df_place_porte = df_immat.select(["nbplaces", "nbportes"])
df_prix = df_immat.select(["prix"])
df_puissance = df_immat.select(["puissance"])

# 3. Convertir le DataFrame PySpark en DataFrame Pandas
df_place_porte= df_place_porte.toPandas()
df_prix= df_prix.toPandas()
df_puissance= df_puissance.toPandas()

# 4. Tracer le diagramme à moustaches pour chaque colonne
plt.figure(figsize=(12, 8))
sns.boxplot(data=df_place_porte)
plt.title("Diagramme à moustache pour détecter les valeurs aberrantes places/portes")
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(12, 8))
sns.boxplot(data=df_puissance)
plt.title("Diagramme à moustache pour détecter les valeurs aberrantes puissance")
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(12, 8))
sns.boxplot(data=df_prix)
plt.title("Diagramme à moustache pour détecter les valeurs aberrantes prix")
plt.xticks(rotation=45)
plt.show()

In [ ]:
#Afficher les valeurs suspectes.

# Trouver la valeur maximale de la colonne "prix"
prix_max = df_immat.select(max("prix")).collect()[0][0]

# Filtrer le DataFrame pour afficher les lignes ayant cette valeur
df_max_prix = df_immat.filter(df_immat["prix"] == prix_max)

# Afficher les lignes avec le prix maximum
df_max_prix.show()

# Trouver la valeur maximale de la colonne "prix"
prix_min = df_immat.select(min("prix")).collect()[0][0]

# Filtrer le DataFrame pour afficher les lignes ayant cette valeur
df_min_prix = df_immat.filter(df_immat["prix"] == prix_min)

# Afficher les lignes avec le prix maximum
df_min_prix.show()

## Fusion des tables **client** et **immatriculation**

In [ ]:
#Fusion des tables
df_client_immat= df_client.join(df_immat, on= "immatriculation")

In [ ]:
df_client_immat.show()
df_client.count()

In [ ]:
#Nombre de match
match_count=df_client_immat.count()

print(f"Nombre de correspondances (matchs) : {match_count}")

In [ ]:
#Compter les doublons
df_duplicates = (
    df_client_immat.groupBy("immatriculation")
    .agg(count("*").alias("count"))
    .filter(col("count") > 1)  # Filtrer les doublons (où le compte est > 1)
)

# Afficher les résultats
df_duplicates.count()


In [ ]:
#Analyser les doublons
duplicate_values = (
    df_client_immat.groupBy("immatriculation")
    .agg(count("*").alias("count"))
    .filter(col("count") > 1)  # Filtrer les doublons
    .select("immatriculation")  # Récupérer les valeurs des doublons
)

# Étape 2 : Joindre pour récupérer les lignes correspondantes
df_duplicate_rows = df_client_immat.join(
    duplicate_values, on="immatriculation", how="inner"
)

# Étape 3 : Ordonnancer par 'immatriculation' et afficher une seule ligne
df_duplicate_rows.orderBy("immatriculation").show(10)

In [ ]:
#Supression des doublons
df_client_immat= df_client_immat.dropDuplicates(["immatriculation"])

In [ ]:
df_duplicates = (
    df_client_immat.groupBy("immatriculation")
    .agg(count("*").alias("count"))
    .filter(col("count") > 1)  # Filtrer les doublons (où le compte est > 1)
)

# Afficher les résultats
df_duplicates.count()

In [ ]:
df_client_immat.printSchema()

In [ ]:
# Ajout de la colonne 'categorie' avec des critères précis
df_client_immat= df_client_immat.withColumn(
    "categorie",
    when(
        (col("longueur") == "courte") & (col("puissance") < 100) & (col("prix") < 20000),
        "citadine economique"
    )
    .when(
        (col("longueur") == "courte") & (col("puissance") >= 100) & (col("prix") >= 20000),
        "citadine standard"
    )
    .when(
        (col("longueur").isin("moyenne", "longue")) & (col("nbplaces") >= 5) & (col("prix") < 35000),
        "familiale"
    )
    .when(
        (col("longueur").isin("longue", "tres longue")) & (col("nbplaces") >= 5) & (col("prix") >= 35000),
        "suv/crossover"
    )
    .when(
        (col("puissance") >= 200) & (col("prix") >= 40000),
        "sportive"
    )
    .when(
        (col("prix") >= 50000),
        "luxe"
    )
    .otherwise("autre")
)

In [ ]:
df_client_immat.show()

In [ ]:
value_client = df_client_immat.groupBy("nbenfantacharge").count()
value_client.show()